# Time Series: Investigate Currency Exchange History £ --> €
## Data: https://www.ofx.com/
## Load libraries and read datafile 

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [57]:
fn = 'ofx-GBP-EUR-2.xlsx'
data = pd.read_excel(fn)
df_fx = pd.DataFrame(data)
df_fx.head()

,Date,Conversion
0,2008-05-07,1.270219
1,2008-05-08,1.269209
2,2008-05-09,1.269134
3,2008-05-10,1.269134
4,2008-05-11,1.269134


### Data Pre-processing
#### $\color{blue}{\text{For some conversion rates the digit seems to have shifted by one, e.g}}$

In [58]:
df_fx[13:16]

,Date,Conversion
13,2008-05-20,1.255727
14,2008-05-21,0.125716
15,2008-05-22,1.247786


$\color{blue}{\text{Option 1: Delete all values < 0.5}}$

In [59]:
df_fx_option1 = df_fx[df_fx.Conversion >0.5]
print(df_fx.shape)
print(df_fx_option1.shape)

(3628, 2)
(3258, 2)


$\color{blue}{\text{Option 2 (with which we will continue): Multiply all values < 0.5 with 10}}$

In [60]:
def multi(x):
    if x < 0.5:
        return(x * 10)
    else:
        return x
    
df_fx['Conversion_op2'] = df_fx['Conversion'].apply(multi)
    

In [61]:
df_fx[13:16]

,Date,Conversion,Conversion_op2
13,2008-05-20,1.255727,1.255727
14,2008-05-21,0.125716,1.257160
15,2008-05-22,1.247786,1.247786


$\color{red}{\text{!!! For some values, even this operation leaves the value below 1.}}$

In [62]:
df_fx[df_fx['Conversion_op2'] < 1].head()

,Date,Conversion,Conversion_op2
113,2008-08-28,0.012473,0.12473
302,2009-03-05,0.011221,0.11221
434,2009-07-15,0.011684,0.11684
435,2009-07-16,0.011649,0.11649
680,2010-03-18,0.011151,0.11151


$\color{red}{\text{These will be treated as artefacts and deleted from the dataset:}}$

In [63]:
df_fx = df_fx[df_fx.Conversion_op2 > 1]

In [64]:
df_fx.head()

,Date,Conversion,Conversion_op2
0,2008-05-07,1.270219,1.270219
1,2008-05-08,1.269209,1.269209
2,2008-05-09,1.269134,1.269134
3,2008-05-10,1.269134,1.269134
4,2008-05-11,1.269134,1.269134


### Extra columns showing year, month and day separately

In [65]:
df_fx['Year'], df_fx['Month'], df_fx['Day'] = (df_fx['Date'].dt.year, 
                                               df_fx['Date'].dt.month, 
                                               df_fx['Date'].dt.day)

In [66]:
df_fx.head()

,Date,Conversion,Conversion_op2,Year,Month,Day
0,2008-05-07,1.270219,1.270219,2008,5,7
1,2008-05-08,1.269209,1.269209,2008,5,8
2,2008-05-09,1.269134,1.269134,2008,5,9
3,2008-05-10,1.269134,1.269134,2008,5,10
4,2008-05-11,1.269134,1.269134,2008,5,11


## Split dataframe into 2, i.e. pre- and post-brexit 

In [67]:
date_of_ref = '2016-06-23' # Date of Referendum
df_fx_pre = df_fx[df_fx['Date'] <= date_of_ref]
df_fx_post = df_fx[df_fx['Date'] > date_of_ref]

In [68]:
print(df_fx_pre.tail())
print(df_fx_post.head())

           Date  Conversion  Conversion_op2  Year  Month  Day
2949 2016-06-19    1.273384        1.273384  2016      6   19
2950 2016-06-20    1.273338        1.273338  2016      6   20
2951 2016-06-21    1.296336        1.296336  2016      6   21
2952 2016-06-22    1.302646        1.302646  2016      6   22
2953 2016-06-23    1.297159        1.297159  2016      6   23
           Date  Conversion  Conversion_op2  Year  Month  Day
2954 2016-06-24    1.305769        1.305769  2016      6   24
2955 2016-06-25    1.228998        1.228998  2016      6   25
2956 2016-06-26    1.230365        1.230365  2016      6   26
2957 2016-06-27    1.230365        1.230365  2016      6   27
2958 2016-06-28    1.196887        1.196887  2016      6   28


In [69]:
# Analysis to be done